In [7]:
import workFactory
import csv
import sys
import re
import os
import pandas as pd

from tqdm import tqdm
from random import randint
from argparse import ArgumentParser
from sklearn.model_selection import train_test_split

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [2]:
def write_df_to_tsv(path, dataframe, WORK_DIR):
    try:
        os.makedirs(WORK_DIR)
    except FileExistsError:
        print("folder exist")
        
    with open(path, 'w+', encoding='utf-8') as writer:
        tsv_writer = csv.writer(writer, delimiter='\t')
        for i, row in tqdm(dataframe.iterrows()):
            tsv_writer.writerow([row['label'], row['content']])

In [26]:
def findFirstMatchParagraph(work, label, pos, neg):
    labels = workFactory.load_tsv_label('./numeric_label.tsv')
    match_list = labels[int(label)-1]['name']
    sep = re.split("。", work['content'])
    posit = -1
    conb = ''
    for i, sen in enumerate(sep):
        for match_word in match_list:
            if match_word in sen:
                posit = i
                break
    if posit > 0:
        cnt = 1
        while 1:
            conb = '。'.join(sep[posit-cnt:posit+cnt])
            if len(conb) > 512:
                break
            cnt += 1
        if work['label'] == 0:
            label = '01'
            neg += 1
        else:
            label = '10'
            pos += 1
    else:
        conb = '。'.join(sep[:100])
        conb = conb[:512]
        if work['label'] == 0:
            label = '01'
        else:
            label = '10'
    
    
    return {"label": label, "content": conb}, pos, neg

In [5]:
DATASET = 'full'
LABEL = '42'
LABEL_PATH = './numeric_label.tsv'
PATH = f'./tsv/{DATASET}/{LABEL}'
WORK_DIR = f'./tsv/first-match/{LABEL}'

In [8]:
fp_works = []
pos_match_count = 0
neg_match_count = 0

train = workFactory.load_tsv_dataset(f'{PATH}/train.tsv')
valid = workFactory.load_tsv_dataset(f'{PATH}/dev.tsv')
test = workFactory.load_tsv_dataset(f'{PATH}/test.tsv')
works = train + valid + test

In [9]:
del(train)
del(valid)
del(test)

In [27]:
for work in works[:5]:
        par, pos_match_count, neg_match_count = findFirstMatchParagraph(work, LABEL, pos_match_count, neg_match_count)
        print(par)
        fp_works.append(par)

{'label': '10', 'content': '一言でいうならアイツは不死身なんだよ」『ふ、不死身！？』相当驚いたのか、電話越しのソフィアの声が大きくなった。思わず顔を背けた雫はスマホを落としてしまう。「急に大声出さないでよソフィアさん……。このスマホ、変えたばかりなんだからさ」『す、すいません……。えっ、でも不死身って……。黒川君と同じなんですか？』「いや、オレのは神機によるものだけどアイツのはなんていうか……。体質？みたいな」『体質……ですか？』「もう今日はこれ以上詳しい説明する気はないけど、『異能精霊』っていう精霊がいるんだよ。なんていうか普通にはありえない、妙な力を持った精霊が」『はあ……』電話の向こうの声は理解できていないようなトーンだった。「その能力でアイツの身体は不死身で出来てる。オレの心臓神機みたいに時間の決まりが無い、いくらでも再生出来る身体でね」『それって……。無敵なんじゃないですか！？』「そうだよ。だからこっちが疲れるだけだから相手にしたくないんだよね」雫は慣れた手つきでコントローラーをカチャカチャと操作しながら会話をしている。『……復活するときっていつもあんなに苦しそうなんですか？』一拍置いてソフィアがそう訊ねてきた。「いや、普段はケロっとした様子ですぐに元通りになるんだけどね。確かに今日は様子がおかしかったなぁ」『そう、なんですか……。……あの、黒川君』「どしたの？」『私、テトラさんとお友達になったんですかね？』「なんでオレに聞くのさ……」『だ、だって……'}
{'label': '01', 'content': 'そしてアイテムを売り払ってひと財産築き上げたら……いつか親父やお袋にもっと贅沢させてやるわ｣彼がそう宣言すると、仲間の二人がヒューヒューと口笛を吹いたり、手を叩いたりしながら喝采を上げた。｢マルティンだけじゃ頼りないですからね。天才魔導士であるこのアヤ・ブリュネ様がお供してあげます！｣｢よっしゃ、そんじゃあこのオレ、ブライアン・パウエルも力を貸してやるぜ！｣｢いらねーよお前らなんて……鬱陶しいんだよ……｣｢馬鹿野郎、本当は嬉しいくせによ！｣｢素直じゃない人は嫌いですよ！｣笑い合いながら互いを小突き合う三人。(うん、この三人なら……心配ないかも)東邦帝国のユキムラにプロポーズして彼との間に子供をもうけたミリアム。その娘のア

In [26]:
fp_works = []
for work in tqdm(pos_works):
    par = findFirstMatchParagraph(work)
    if len(par)!=0:
        #print(par)
        fp_works.append(par)

In [44]:
final_works = []
fp_length = len(fp_works)
for work in fp_works:
    dat = {"label": "10", "content": work}
    final_works.append(dat)
    
neg_works = neg_works[:fp_length]
for work in neg_works:
    cnt = work['content']
    if len(cnt)<512:
        dat = {"label": "01", "content": cnt}
    else:
        rnd = randint(0, len(cnt)-512)
        dat = {"label": "01", "content": cnt[rnd:rnd+512]}
    final_works.append(dat)

In [45]:
print(len(final_works))
print(final_works[1700])
print(final_works[1701])

3402
{'label': '10', 'content': '「成る程な......と言うことは裏を返せば暴走した時に複数人で対処が必要な程の能力って事だな？「大正解貴方もああなりたくなかったら何かしら策を講じなさい  幻坂はフィールドを指差すそこには獅子頭の大男に胴体を片手で掴まれ、今にも食べられそうになっている祈祷先輩の姿があったよく見れば白目を剥いて失神しており、ジャッジが入っていた「......肝に命じておくよ  あまりにも規格外過ぎる相手を前に、思索することすら無駄なのではないか......そんな思いを抱えながら頭を巡らせるのであった'}
{'label': '01', 'content': 'を見ることはないのだから。警戒するのも無理はない。「この方は、アマゾネスのリアさんです。その背中に背負われているのは先輩魔導士のミリアム先輩。二人とも私の仲間です」「──仲間ね……」サヤは警戒を解こうとしない。動いたのはリアだった。彼女はミリアムをゆっくりと床に下ろすと、サヤの前に進み出る。「あなたがティナの知り合いで七天のサヤって人？」「……そうだけど？」返答を聞いたリアはうんうんと頷くと腰をかがめてニヤリと笑った。刹那、リアがまとう雰囲気がガラリと変わった。「──やっと見つけた！」(これは……殺気!? でもどうして!?)「危ないっ──!?」私が警告するよりも早く──サヤが反応するよりも早く、リアが目にも止まらぬ速さで前方に飛び出す。右手にはナイフが握られており、それが真っすぐにサヤの胸元に吸い込まれていく。──ドッ！衝撃をともなった鈍い音が響く。と同時に真っ赤な血しぶきが舞った。「えっ……？」私は目の前で起こったことが信じられなかった。リアの腕は手首くらいまでサヤの体内に埋まっており、サヤの背中からはナイフの先端が飛び出している。傷口からは絶え間なく血が吹き出し、たちまち二人の足元に赤い水たまりを形成した'}


In [48]:
final_df = pd.DataFrame(final_works)
train, valid_test = train_test_split(final_df, test_size=0.2)
valid, test = train_test_split(valid_test, test_size=0.5)

print(f'Train: {len(train)} / Valid: {len(valid)} / Test: {len(test)}')

del(final_df)
del(valid_test)

Train: 2721 / Valid: 340 / Test: 341


In [53]:
write_df_to_tsv(f'{WORK_DIR}/train.tsv', train, WORK_DIR)
write_df_to_tsv(f'{WORK_DIR}/dev.tsv', valid, WORK_DIR)
write_df_to_tsv(f'{WORK_DIR}/test.tsv', test, WORK_DIR)

2721it [00:00, 19285.57it/s]


folder exist


340it [00:00, 18649.64it/s]


folder exist


341it [00:00, 19121.86it/s]
